In [1]:
!pip install youtube_transcript_api
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 4.4 MB/s eta 0:00:00


In [5]:
pip install langchain langchain-community

In [3]:
%pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.7 MB/s eta 0:00:00


In [28]:
%pip install -qU langchain-openai

In [44]:
from youtube_transcript_api import YouTubeTranscriptApi
import re
import contractions # pip install contractions
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
import gradio as gr
from groq import Groq
import os, getpass

### Extracting a YouTube transcript from a video URL

In [6]:
def get_video_id(url):
  match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11})", url)
  return match.group(1) if match else None

In [7]:
video_id = get_video_id("https://youtu.be/N_OOfkEWcOk?si=kNUY-0fRKTttqJ7n")

In [8]:
yt = YouTubeTranscriptApi()
transcript = yt.fetch(video_id=video_id)

In [9]:
transcript

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text="Hi, my name is Rohan and I'm a senior solutions\xa0\narchitect working on Enterprise Generative AI at\xa0\xa0", start=4.08, duration=5.44), FetchedTranscriptSnippet(text="NVIDIA. Large language models, or LLMs, are taking\xa0\nthe world by storm. And now, they're easier to\xa0\xa0", start=9.52, duration=7.16), FetchedTranscriptSnippet(text="develop and deploy than ever, even without needing\xa0\nto host your own GPU infrastructure. Today, I'll\xa0\xa0", start=16.68, duration=6.08), FetchedTranscriptSnippet(text='be showing you how to design an enterprise-grade\xa0\nretrieval augmented generation, or RAG, pipeline\xa0\xa0', start=22.76, duration=6.48), FetchedTranscriptSnippet(text="using NVIDIA's AI Foundation models. Since all of\xa0\nthe embedding and generation happens on NVIDIA AI\xa0\xa0", start=29.24, duration=5.96), FetchedTranscriptSnippet(text="Foundation endpoints, no local GPU is required.\xa0\nIf you'd like to exper

In [10]:
text_list = [snippet.text for snippet in transcript.snippets]
text_list

["Hi, my name is Rohan and I'm a senior solutions\xa0\narchitect working on Enterprise Generative AI at\xa0\xa0",
 "NVIDIA. Large language models, or LLMs, are taking\xa0\nthe world by storm. And now, they're easier to\xa0\xa0",
 "develop and deploy than ever, even without needing\xa0\nto host your own GPU infrastructure. Today, I'll\xa0\xa0",
 'be showing you how to design an enterprise-grade\xa0\nretrieval augmented generation, or RAG, pipeline\xa0\xa0',
 "using NVIDIA's AI Foundation models. Since all of\xa0\nthe embedding and generation happens on NVIDIA AI\xa0\xa0",
 "Foundation endpoints, no local GPU is required.\xa0\nIf you'd like to experiment with open-source LLMs\xa0\xa0",
 'accelerated by the latest NVIDIA GPUs, check out\xa0\nthese endpoints. We host a collection of models\xa0\xa0',
 'you can interact with through our UI. NVIDIA NeVA,\xa0\nthe NEMO visual and language assistant model,\xa0\xa0',
 'is a great example. We can use NeVA for multimodal\xa0\nimage and language un

In [11]:
transcript_text = " ".join(text_list)#.replace('\xa0', ' ').replace('\n', '')
transcript_text

'Hi, my name is Rohan and I\'m a senior solutions\xa0\narchitect working on Enterprise Generative AI at\xa0\xa0 NVIDIA. Large language models, or LLMs, are taking\xa0\nthe world by storm. And now, they\'re easier to\xa0\xa0 develop and deploy than ever, even without needing\xa0\nto host your own GPU infrastructure. Today, I\'ll\xa0\xa0 be showing you how to design an enterprise-grade\xa0\nretrieval augmented generation, or RAG, pipeline\xa0\xa0 using NVIDIA\'s AI Foundation models. Since all of\xa0\nthe embedding and generation happens on NVIDIA AI\xa0\xa0 Foundation endpoints, no local GPU is required.\xa0\nIf you\'d like to experiment with open-source LLMs\xa0\xa0 accelerated by the latest NVIDIA GPUs, check out\xa0\nthese endpoints. We host a collection of models\xa0\xa0 you can interact with through our UI. NVIDIA NeVA,\xa0\nthe NEMO visual and language assistant model,\xa0\xa0 is a great example. We can use NeVA for multimodal\xa0\nimage and language understanding through the UI,\

In [12]:

def normalize_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Expand contractions
    text = contractions.fix(text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [13]:
transcript_text = normalize_text(transcript_text)
transcript_text

'hi, my name is rohan and i am a senior solutions architect working on enterprise generative ai at nvidia. large language models, or llms, are taking the world by storm. and now, they are easier to develop and deploy than ever, even without needing to host your own gpu infrastructure. today, i will be showing you how to design an enterprise-grade retrieval augmented generation, or rag, pipeline using nvidia\'s ai foundation models. since all of the embedding and generation happens on nvidia ai foundation endpoints, no local gpu is required. if you would like to experiment with open-source llms accelerated by the latest nvidia gpus, check out these endpoints. we host a collection of models you can interact with through our ui. nvidia neva, the nemo visual and language assistant model, is a great example. we can use neva for multimodal image and language understanding through the ui, but we can also access nvidia endpoints in our applications through api calls. let us build a local rag a

### Creaing a function to extract and clean a YouTube transcript from a video URL

In [14]:
def get_video_transcript(url):
  match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11})", url)
  video_id = match.group(1) if match else None
  yt = YouTubeTranscriptApi()
  transcript = yt.fetch(video_id)
  text_list = [snippet.text for snippet in transcript.snippets]
  return normalize_text( " ".join(text_list))

In [15]:
transcript_text = get_video_transcript("https://youtu.be/N_OOfkEWcOk?si=QMAEhom_hQP7qe_a")
transcript_text

'hi, my name is rohan and i am a senior solutions architect working on enterprise generative ai at nvidia. large language models, or llms, are taking the world by storm. and now, they are easier to develop and deploy than ever, even without needing to host your own gpu infrastructure. today, i will be showing you how to design an enterprise-grade retrieval augmented generation, or rag, pipeline using nvidia\'s ai foundation models. since all of the embedding and generation happens on nvidia ai foundation endpoints, no local gpu is required. if you would like to experiment with open-source llms accelerated by the latest nvidia gpus, check out these endpoints. we host a collection of models you can interact with through our ui. nvidia neva, the nemo visual and language assistant model, is a great example. we can use neva for multimodal image and language understanding through the ui, but we can also access nvidia endpoints in our applications through api calls. let us build a local rag a

### Building the model using GROQ API

In [25]:
# Safe way to set the API key
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [48]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=1,
    api_key=os.environ["GROQ_API_KEY"]
)

In [51]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert summarizer that generates clear, accurate, and concise summaries of YouTube videos based on their transcripts. \
    Do not include titles, bullet points, or formatting only return plain text."),
    ("human", "Please summarize the following YouTube video transcript. Transcript: {transcript}")
])

RunnableVideoTranscript = RunnableLambda(get_video_transcript)
parser = StrOutputParser()
chain = RunnableVideoTranscript | template | llm | parser
response = chain.invoke("https://youtu.be/N_OOfkEWcOk?si=QMAEhom_hQP7qe_a")

In [52]:
response

"Rohan, a senior solutions architect at NVIDIA, is discussing the design of an enterprise-grade retrieval augmented generation, or RAG, pipeline using NVIDIA's AI Foundation Models. He explains that users can experiment with open-source LLMs accelerated by the latest NVIDIA GPUs via NVIDIA's AI Foundation Endpoints, which don't require local GPU infrastructure. \n\nTo build a local RAG application, four main components are needed: a custom data loader, a text embedding model, a vector database, and a large language model for response generation. Rohan demonstrates how to use NVIDIA's AI Foundation Endpoints through the LangChain connector to perform these tasks. \n\nThe steps involve creating an API key through the NVIDIA NGC, building a chat UI with Streamlit, and integrating the custom data connector, text embedding model, and vector database. The final RAG pipeline is then brought together into a user-friendly interface that retrieves relevant documents and responds to user prompts.

### Building the model using openAI API

In [53]:
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [54]:
llm = ChatOpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1",  # Groq endpoint
    model="openai/gpt-oss-20b",
    temperature=0.3
)

In [70]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert summarizer that generates clear, accurate, and concise summaries of YouTube videos based on their transcripts. \
    Do not include titles, bullet points, or formatting only return plain text."),
    ("human", "Please summarize the following YouTube video transcript. Transcript: {transcript}")
])

RunnableVideoTranscript = RunnableLambda(get_video_transcript)
parser = StrOutputParser()
chain = RunnableVideoTranscript | template | llm | parser
response = chain.invoke("https://youtu.be/N_OOfkEWcOk?si=QMAEhom_hQP7qe_a")
response

'Rohan, a senior solutions architect at NVIDIA, explains how to build an enterprise‑grade retrieval‑augmented generation (RAG) pipeline using NVIDIA’s AI Foundation models. He outlines the four core components: a custom data loader to segment documents, an embedding model that converts text chunks into vectors, a vector database (using FAISS) to store those embeddings, and a large language model to generate responses. Rohan shows how to use LangChain connectors to access NVIDIA’s hosted endpoints, and how to assemble the pipeline in under five minutes with about 100 lines of Python, including a Streamlit user interface for uploading documents, embedding them, storing the vectors, and querying the LLM. He notes that the entire process requires no local GPU, that NVIDIA offers free access for up to 10,000 API transactions, and that the open‑source connectors and example code on GitHub enable developers to quickly prototype and expand their own RAG applications.'

### Creating a Web Interface for our Youtube Summarizer

In [61]:
def create_web_interface(chain):
    def generate_response(user_input):
        return chain.invoke(user_input)

    with gr.Blocks() as app:
        with gr.Row():
            text_input = gr.Textbox(label="User Message", placeholder="Ready to summarize? Paste a YouTube link.")

        output_area = gr.Textbox(label="AI Response", interactive=False, lines=10)

        text_input.submit(generate_response, inputs=text_input, outputs=output_area)

    return app

In [62]:
app = create_web_interface(chain)
app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://64eaace08382c1b30e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
